# Sprint16課題 論文読解入門

以下の論文を読み問題に答えてください。CNNを使った物体検出（Object Detection）の代表的な研究です。

[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99

https://arxiv.org/pdf/1506.01497.pdf



## 問題

それぞれについてJupyter Notebookにマークダウン形式で記述してください。



### (1) 物体検出の分野にはどういった手法が存在したか。  
SPPnet, Fast R-CNN,R-CNN

#### 参照
**Abstract**

State-of-the-art object detection networks depend on region proposal algorithms to hypothesize object locations. Advances like SPPnet [1] and Fast R-CNN [2] have reduced the running time of these detection networks, exposing region proposal computation as a bottleneck.

### (2) Fasterとあるが、どういった仕組みで高速化したのか。  
Fast R-CNNではRegion ProposalにSelective Serchを使用しており、そこがボトルネックとなっていたが、Faster R-CNNではRegionProposalもCNN化したRPN(Region Proposal Networks)を構築し、end-to-endで学習を行うことによる高速化を可能とした。

#### 参照
**1. INTRODUTION**  

In this paper, we show that an algorithmic change— computing proposals with a deep convolutional neural network—leads to an elegant and effective solution where proposal computation is nearly cost-free given the detection network’s computation. To this end, we introduce novel Region Proposal Networks (RPNs) that share convolutional layers with state-of-the-art object detection networks [1], [2]. By sharing convolutions at test-time, the marginal cost for computing proposals is small (e.g., 10ms per image).

Our observation is that the convolutional feature maps used by region-based detectors, like Fast RCNN, can also be used for generating region proposals. On top of these convolutional features, we construct an RPN by adding a few additional convolutional layers that simultaneously regress region bounds and objectness scores at each location on a regular grid. The RPN is thus a kind of fully convolutional network (FCN) [7] and can be trained end-to-end specifically for the task for generating detection proposals.

### (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。
物体検出手法の違い。
- One-Stage  
畳み込みの特徴マップ上で領域の推定とクラスの分類を同時に推定する手法。論文上ではOverFeatを紹介している。
- Two-Stage  
領域候補の推定を行なった上で領域の特徴からクラス分類を行う手法。Faster R-CNNなど、R-CNN系に利用されている。精度の高い検出を実現することができるが、One-Stageより速度が遅くなっていることに注意する必要がある。




#### 参照

**3.1 Region Proposal Networks**


A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score.3 We model this process with a fully convolutional network [7], which we describe in this section. Because our ultimate goal is to share computation with a Fast R-CNN object detection network [2], we assume that both nets share a common set of convolutional layers. In our experiments, we investigate the Zeiler and Fergus model [32] (ZF), which has 5 shareable convolutional layers and the Simonyan and Zisserman model [3] (VGG-16), which has 13 shareable convolutional layers.  
To generate region proposals, we slide a small network over the convolutional feature map output by the last shared convolutional layer. This small network takes as input an n × n spatial window of the input convolutional feature map. Each sliding window is mapped to a lower-dimensional feature (256-d for ZF and 512-d for VGG, with ReLU [33] following). This feature is fed into two sibling fullyconnected layers—a box-regression layer (reg) and a box-classification layer (cls). We use n = 3 in this paper, noting that the effective receptive field on the input image is large (171 and 228 pixels for ZF and
VGG, respectively). This mini-network is illustrated at a single position in Figure 3 (left). Note that because the mini-network operates in a sliding-window fashion, the fully-connected layers are shared across all spatial locations. This architecture is naturally implemented with an n×n convolutional layer followed by two sibling 1 × 1 convolutional layers (for reg and cls, respectively).

### (5) RoIプーリングとは何か。  

maxプーリングを利用して、任意のサイズの入力から固定サイズの特徴マップを得ること。  
SPPnetで利用されているspatial pyramid pooling層を基にしたもので、一層のみのピラミッドレベルになっている。  
Roiプーリングが利用される以前でのR-CNNでは、大量のregion proposalを0からCNNにかける必要があり、非常に低速であったが、この固定サイズの特徴マップを抽出する手法を取り入れることにより、処理の高速化を実現した。


#### 参照
- [Fast R-CNN](https://arxiv.org/pdf/1504.08083.pdf)  

**2.1. The RoI pooling layer**

The RoI pooling layer uses max pooling to convert the features inside any valid region of interest into a small feature map with a fixed spatial extent of H × W (e.g., 7 × 7), where H and W are layer hyper-parameters that are independent of any particular RoI. 
In this paper, an RoI is a rectangular window into a conv feature map. Each RoI is defined by a four-tuple (r, c, h, w) that specifies its top-left corner (r, c) and its height and width (h, w).

RoI max pooling works by dividing the h × w RoI window into an H × W grid of sub-windows of approximate size h/H × w/W and then max-pooling the values in each sub-window into the corresponding output grid cell. Pooling is applied independently to each feature map channel, as in standard max pooling. The RoI layer is simply the special-case of the spatial pyramid pooling layer used in SPPnets [11] in which there is only one pyramid level. We use the pooling sub-window calculation given in [11].

### (6) Anchorのサイズはどうするのが適切か  
入力画像における対象物のスケールと縦横比を考慮して決定することが適切である。論文では、3つのスケールと3つの縦横比を使用し、3 * 3 = 9のAnchorを生成している。  
例えば車などの検出であれば、横長のAnchorを準備する必要がある。

#### 参照
An anchor is centered at the sliding window in question, and is associated with a scale and aspect ratio (Figure 3, left). 
By default we use 3 scales and 3 aspect ratios, yielding k = 9 anchors at each sliding position. For a convolutional feature map of a size W × H (typically ∼2,400), there are W Hk anchors in total.

### (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

    

- PASCAL VOC 2007 test set  
Selective Serch(SS)とRPN+VGGの手法にて比較を行う。同一学習データ(PASCAL VOC 2007と2012)を利用してmean Average Precision(mAP)が3.2%上昇した。

- PASCAL VOC 2012 test set  
Selective Serch(SS)とRPN+VGGの手法にて比較を行う。同一学習データ(PASCAL VOC 2007と2012)を利用してmean Average Precision(mAP)が2.0%上昇した。

- PASCAL VOC  
速度の比較。SS + Fast R-CNNでは1830ミリ秒で予測処理完了したのが、RPN + Fast R-CNNでは198ミリ秒で完了している。

- MS COCO
Fast R-CNNとFaster R-CNNに精度比較を行っている。MS COCOのテストデータに対して、「mAP@.5」、「mAP@[.5, .95]」にてそれぞれ2.0〜5.0%程度の精度改善が得られた。

### (8) （アドバンス）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

### Mask R-CNN
https://arxiv.org/pdf/1703.06870.pdf

- Mask R-CNNは、Faster R-CNNに対して各対象領域における予測分割を行うmaskの工程を加えた手法である  
Faster R-CNN [36] advanced this stream by learning the attention mechanism with a Region Proposal Network (RPN). Faster R-CNN is flexible and robust to many follow-up improvements (e.g., [38, 27, 21]), and is the current leading framework in several benchmarks.


- Faster R-CNNにて利用されている　ROIプーリングでは位置を合わせる際に誤差が発生し、ピクセル単位での制度にネガティブな影響を与えてしまっている。 
These quantizations introduce misalignments between the RoI and the extracted features. While this may not impact classification, which is robust to small translations, it has a large negative effect on predicting pixel-accurate masks. 
